 ## Channel Independence Patch Time Series Transformer

 Channel Independence PatchTST model - an efficient design of Transformer-based models for multivariate time series forecasting and self-supervised representation learning - is demonstrated in the following diagram. It is based on two key components:

   - segmentation of time series into subseries-level patches which are served as input tokens to Transformer;

   - channel-independence where each channel contains a single univariate time series that shares the same embedding and Transformer weights across all the series.

 Patching design naturally has three-fold benefit: local semantic information is retained in the embedding; computation and memory usage of the attention maps are quadratically reduced given the same look-back window; and the model can attend longer history.

 Channel independence allows each time series to have its own embedding and attention maps while sharing the same model parameters across different channels.

 <div> <img src="./figures/patchTST.png" alt="Drawing" style="width: 600px;"/></div>

In [1]:
import pandas as pd

from tsfmservices.toolkit.dataset import PretrainDFDataset
from transformers import (
    PatchTSTConfig,
    PatchTSTForMaskPretraining,
    Trainer,
    TrainingArguments,
)

 ## Load and prepare datasets

 For the dataset of interest, you must supply the names of the following columns:

 - timestamp_column: column name containing timestamp information, use None if there is no such column
 - id_columns: List of column names specifying the IDs of different time series. If no ID column exists, use []
 - forecast_columns: List of columns to be modeled

 The data is first loaded into a Pandas dataframe and then converted the appropriate torch dataset needed
 for training. Finaly, the data is split into training and evaluation datasets.

In [2]:
timestamp_column = "date"
id_columns = []
forecast_columns = ["HUFL", "HULL", "MUFL", "MULL", "LUFL", "LULL"]

context_length = 16  # 512

data = pd.read_csv(
    "https://raw.githubusercontent.com/zhouhaoyi/ETDataset/main/ETT-small/ETTh1.csv",
    parse_dates=[timestamp_column],
)
print(data.head())

# to do: split data
# need utility here, group sensitive splitting should be done
train_data = data.iloc[: 12 * 30 * 24,].copy()
eval_data = data.iloc[
    12 * 30 * 24 - context_length : 12 * 30 * 24 + 4 * 30 * 24,
].copy()


train_dataset = PretrainDFDataset(
    train_data,
    timestamp_column=timestamp_column,
    id_columns=id_columns,
    input_columns=forecast_columns,
    context_length=context_length,
)
eval_dataset = PretrainDFDataset(
    eval_data,
    timestamp_column=timestamp_column,
    id_columns=id_columns,
    input_columns=forecast_columns,
    context_length=context_length,
)

                 date   HUFL   HULL   MUFL   MULL   LUFL   LULL         OT
0 2016-07-01 00:00:00  5.827  2.009  1.599  0.462  4.203  1.340  30.531000
1 2016-07-01 01:00:00  5.693  2.076  1.492  0.426  4.142  1.371  27.787001
2 2016-07-01 02:00:00  5.157  1.741  1.279  0.355  3.777  1.218  27.787001
3 2016-07-01 03:00:00  5.090  1.942  1.279  0.391  3.807  1.279  25.044001
4 2016-07-01 04:00:00  5.358  1.942  1.492  0.462  3.868  1.279  21.948000


 ## Configure the PatchTST model

 Describe parameters here.

In [3]:
config = PatchTSTConfig(
    num_input_channels=len(forecast_columns),
    context_length=context_length,
    patch_length=12,
    stride=12,
    mask_ratio=0.4,
    # standardscale=None,  # 'bysample'
    d_model=16,  # 128,
    encoder_attention_heads=2,  # 16,
    encoder_layers=2,  # 6,
    encoder_ffn_dim=16,  # 512,
    dropout=0.2,
    head_dropout=0.2,
)
pretraining_model = PatchTSTForMaskPretraining(config)

In [4]:
training_args = TrainingArguments(
    output_dir="./checkpoint/pretrain",
    # logging_steps = 100,
    num_train_epochs=3,
    evaluation_strategy="epoch",
    save_strategy="epoch",
    # eval_steps = 100,
    save_total_limit=5,
    logging_strategy="epoch",
    load_best_model_at_end=True,
    use_mps_device=True,
    max_steps=10,
    label_names=["past_values"],
)
pretrainer = Trainer(
    model=pretraining_model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=eval_dataset,
)

pretrainer.train()

/Users/wmgifford/opt/miniconda3/envs/test_ogv/lib/python3.10/site-packages/transformers/training_args.py:1812: UserWarning: `use_mps_device` is deprecated and will be removed in version 5.0 of 🤗 Transformers.`mps` device will be used by default if available similar to the way `cuda` device is used.Therefore, no action from user is required. 
  warnings.warn(
/Users/wmgifford/opt/miniconda3/envs/test_ogv/lib/python3.10/site-packages/transformers/models/patchtst/modeling_patchtst.py:299: UserWarning: torch.sort is supported by MPS on MacOS 13+, please upgrade. Falling back to CPU (Triggered internally at /Users/runner/work/pytorch/pytorch/pytorch/aten/src/ATen/native/mps/operations/Sort.mm:30.)
  ids_shuffle = torch.argsort(noise, dim=-1)  # ascend: small is keep, large is remove


Epoch,Training Loss,Validation Loss
0,0.929200,0.917777


TrainOutput(global_step=10, training_loss=0.9291750907897949, metrics={'train_runtime': 11.1571, 'train_samples_per_second': 7.17, 'train_steps_per_second': 0.896, 'total_flos': 176025600.0, 'train_loss': 0.9291750907897949, 'epoch': 0.01})

In [5]:
pretrainer.save_model("model/pretrained")